# ResNet-50을 활용해봅시다.

In [1]:
import random
import time
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from PIL import Image
from tqdm import tqdm

plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

random.seed(42)

cpu


In [2]:
transform_train = (
    transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.Grayscale(num_output_channels=3),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    ),
)

transform_test = (
    transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.Grayscale(num_output_channels=3),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    ),
)

In [3]:
import torchvision
train_dataset = torchvision.datasets.MNIST(root="./data", 
                                           train=True, 
                                           download=True, 
                                           transform=transform_train)
test_dataset = torchvision.datasets.MNIST(root="./data", 
                                           train=False, 
                                           download=True, 
                                           transform=transform_test)

100%|██████████| 9.91M/9.91M [00:14<00:00, 688kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 139kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.32MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.06MB/s]


In [4]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
for _ in train_loader:
    pass

TypeError: 'tuple' object is not callable

In [6]:
# ResNet-50 모델 로드 및 전이학습 설정
def create_resnet50_model(num_classes=10, pretrained=True):
    if pretrained:
        model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1)
    else:
        model = torchvision.models.resnet50(weights=None)

    for param in model.parameters():
        param.requires_grad = False
    
    num_feat = model.fc.in_features
    model.fc = nn.Linear(num_feat, num_classes)
    return model

In [7]:
def train_model(model, train_loader, criterion, optimizer):
    model.train()
    loss = 0.0
    correct = 0
    total = 0    
    for _, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        outputs = model(data)        
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        loss += loss.item()
        _, preds = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (preds == target).sum().item()    
    avg_loss = loss / len(train_loader)
    acc = 100 * correct / total    
    return avg_loss, acc

In [8]:
model = create_resnet50_model()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.002)
num_epoch = 10
loss, acc = train_model(model, train_loader, criterion, optimizer)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\user/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:02<00:00, 36.9MB/s]


TypeError: 'tuple' object is not callable